In [36]:
import pickle
import os
import pandas as pd
from matplotlib import pyplot as plt
os.chdir(os.path.expanduser('~/wcEcoli/'))
# noinspection PyUnresolvedReferences
import numpy as np
import plotly.graph_objects as go
from models.ecoli.analysis import cohortAnalysisPlot
from wholecell.analysis.analysis_tools import (exportFigure,
	read_bulk_molecule_counts, read_stacked_bulk_molecules, read_stacked_columns)
from wholecell.io.tablereader import TableReader
from sklearn.metrics import r2_score
import plotly.graph_objects as go
import glob
import plotly.express as px

read in the priority substrate data

In [6]:
# read in the priority protease data: 
priority_substrates = "models/ecoli/analysis/local_notebooks/protease_substrate_relationships/data/priority_protease_assignments_0.tsv"
priority_substrates = pd.read_csv(priority_substrates, sep='\t', skiprows=1)
priority_substrates

,id,protease_assignment,ClpP,Lon,HslV,Unexplained
0,EG10156-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000
1,MDLB-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000
2,EG10927-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000
3,EG10618-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000
4,AERGLYC3PDEHYDROG-MONOMER,ClpP only,0.993731,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
77,EG12690-MONOMER,"Additive: ClpP, Lon, HslV",0.277972,0.277364,0.378556,0.066108
78,PD03831,"Additive: ClpP, Lon",0.519958,0.403122,0.000000,0.076920
79,AROK-MONOMER,"Additive: ClpP, Lon, HslV",0.319436,0.429511,0.167152,0.083901
80,G7263-MONOMER,"Additive: ClpP, Lon, HslV",0.183526,0.206054,0.520932,0.089488


read in the half life data from CLNE1 (original model)

In [7]:
# read in the CLNE1 half life data: 
CLNE1_HLs = "out/CLNE1/kb_plot_out/protein_half_lives.tsv"
CLNE1_HLs = pd.read_csv(CLNE1_HLs, sep='\t')
CLNE1_HLs

,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000019,600.0,N_end_rule
1,1-PFK-MONOMER,0.000019,600.0,N_end_rule
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,0.000019,600.0,N_end_rule
3,2-ISOPROPYLMALATESYN-MONOMER,0.000019,600.0,N_end_rule
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000019,600.0,N_end_rule
...,...,...,...,...
4305,YTFR-MONOMER,0.000019,600.0,N_end_rule
4306,YTFT-MONOMER,0.000019,600.0,N_end_rule
4307,ZNUA-MONOMER,0.000019,600.0,N_end_rule
4308,ZNUB-MONOMER,0.000019,600.0,N_end_rule


In [9]:
# make a new data frame that contains the monomers that only have a half life of 600 in the half_life_(min) column of CLNE1_HLs:
CLNE1_HLs_600 = CLNE1_HLs[CLNE1_HLs['half_life_(min)'] == 600]
CLNE1_HLs_600

,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000019,600.0,N_end_rule
1,1-PFK-MONOMER,0.000019,600.0,N_end_rule
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,0.000019,600.0,N_end_rule
3,2-ISOPROPYLMALATESYN-MONOMER,0.000019,600.0,N_end_rule
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000019,600.0,N_end_rule
...,...,...,...,...
4305,YTFR-MONOMER,0.000019,600.0,N_end_rule
4306,YTFT-MONOMER,0.000019,600.0,N_end_rule
4307,ZNUA-MONOMER,0.000019,600.0,N_end_rule
4308,ZNUB-MONOMER,0.000019,600.0,N_end_rule


In [10]:
# make a new data frame that contains the monomers that only have a half life of 2 in the half_life_(min) column of CLNE1_HLs:
CLNE1_HLs_2 = CLNE1_HLs[CLNE1_HLs['half_life_(min)'] == 2]
CLNE1_HLs_2
# note, the remianing half lives are Covert lab assigned half lives!

,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source
517,EG10506-MONOMER,0.005776,2.0,N_end_rule
611,EG10765-MONOMER,0.005776,2.0,N_end_rule
2754,G7426-MONOMER,0.005776,2.0,N_end_rule
3293,ISOCIT-LYASE-MONOMER,0.005776,2.0,N_end_rule
3787,OROPRIBTRANS-MONOMER,0.005776,2.0,N_end_rule
3887,PGPPHOSPHAB-MONOMER,0.005776,2.0,N_end_rule
3927,PPENTOMUT-MONOMER,0.005776,2.0,N_end_rule


read in the half life data from CLClim3NE1 (new proposed sort)

In [8]:
# read in the CLClim3NE1 half life data:
CLClim3NE1_HLs = "out/CLClim3NE1/kb_plot_out/protein_half_lives.tsv"
CLClim3NE1_HLs = pd.read_csv(CLClim3NE1_HLs, sep='\t')
CLClim3NE1_HLs

,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000023,501.707893,Gupta_et_al_MS_2024
1,1-PFK-MONOMER,0.000015,766.256894,Gupta_et_al_MS_2024
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,0.000019,600.000000,N_end_rule
3,2-ISOPROPYLMALATESYN-MONOMER,0.000042,276.924332,Gupta_et_al_MS_2024
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000023,493.397050,Gupta_et_al_MS_2024
...,...,...,...,...
4305,YTFR-MONOMER,0.000018,654.149344,Gupta_et_al_MS_2024
4306,YTFT-MONOMER,0.000019,600.000000,N_end_rule
4307,ZNUA-MONOMER,0.000018,638.129701,Gupta_et_al_MS_2024
4308,ZNUB-MONOMER,0.000019,600.000000,N_end_rule


Find the proteins in the CLClim3NE1_HLs data frame that have a half life of 600 in the half_life_(min) column

In [16]:
# make a new data frame from CLCLim3NE1_HLs that contains only the monomers that show up in CLNE1_HLs_600:
CLClim3NE1_HLs_were_600 = CLNE1_HLs_600.merge(CLClim3NE1_HLs, on='monomer_id', how='inner')
# remove columns 2, 3, and 4 from the data frame:
CLClim3NE1_HLs_were_600 = CLClim3NE1_HLs_were_600.drop(columns=['half_life_(min)_x', 'degradation_rate(1/s)_x', 'degradation_rate_source_x'])


CLClim3NE1_HLs_were_600 # 4296
# note that the other 600 min rate in here is the CL_measured_deg_rates_2020 rate for CdsA I think. So dont count that in here since it is not being reassigned. 


,monomer_id,degradation_rate(1/s)_y,half_life_(min)_y,degradation_rate_source_y
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000023,501.707893,Gupta_et_al_MS_2024
1,1-PFK-MONOMER,0.000015,766.256894,Gupta_et_al_MS_2024
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,0.000019,600.000000,N_end_rule
3,2-ISOPROPYLMALATESYN-MONOMER,0.000042,276.924332,Gupta_et_al_MS_2024
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000023,493.397050,Gupta_et_al_MS_2024
...,...,...,...,...
4291,YTFR-MONOMER,0.000018,654.149344,Gupta_et_al_MS_2024
4292,YTFT-MONOMER,0.000019,600.000000,N_end_rule
4293,ZNUA-MONOMER,0.000018,638.129701,Gupta_et_al_MS_2024
4294,ZNUB-MONOMER,0.000019,600.000000,N_end_rule


find the proteins that still are 600 mins in the new model

In [18]:
CLClim3NE1_HLs_still_600 = CLClim3NE1_HLs_were_600[CLClim3NE1_HLs_were_600['degradation_rate_source_y'] == "N_end_rule"]
CLClim3NE1_HLs_still_600 # 1628

,monomer_id,degradation_rate(1/s)_y,half_life_(min)_y,degradation_rate_source_y
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,0.000019,600.0,N_end_rule
19,ABGT-MONOMER,0.000019,600.0,N_end_rule
30,ACPSUB-MONOMER,0.000019,600.0,N_end_rule
33,ACRF-MONOMER,0.000019,600.0,N_end_rule
51,AGAC-MONOMER,0.000019,600.0,N_end_rule
...,...,...,...,...
4285,YPHE-MONOMER,0.000019,600.0,N_end_rule
4287,YRBE-MONOMER,0.000019,600.0,N_end_rule
4289,YRBG-MONOMER,0.000019,600.0,N_end_rule
4292,YTFT-MONOMER,0.000019,600.0,N_end_rule


find the proteins that are no longer 600 mins in the new model

In [20]:
CLClim3NE1_HLs_Clim3 = CLClim3NE1_HLs_were_600[CLClim3NE1_HLs_were_600['degradation_rate_source_y'] == "Gupta_et_al_MS_2024"]
CLClim3NE1_HLs_Clim3 # 2667

,monomer_id,degradation_rate(1/s)_y,half_life_(min)_y,degradation_rate_source_y
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000023,501.707893,Gupta_et_al_MS_2024
1,1-PFK-MONOMER,0.000015,766.256894,Gupta_et_al_MS_2024
3,2-ISOPROPYLMALATESYN-MONOMER,0.000042,276.924332,Gupta_et_al_MS_2024
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000023,493.397050,Gupta_et_al_MS_2024
5,2-OCTAPRENYLPHENOL-HYDROX-MONOMER,0.000015,796.038829,Gupta_et_al_MS_2024
...,...,...,...,...
4288,YRBF-MONOMER,0.000014,823.833163,Gupta_et_al_MS_2024
4290,YTFQ-MONOMER,0.000015,790.418789,Gupta_et_al_MS_2024
4291,YTFR-MONOMER,0.000018,654.149344,Gupta_et_al_MS_2024
4293,ZNUA-MONOMER,0.000018,638.129701,Gupta_et_al_MS_2024


Now, within the new Clim3 rates, find those that are a priority substrate

In [29]:
# make a new column in the CLClim3NE1_HLs_Clim3 data frame that contains the monomer_id's that are priority substrates:

CLClim3NE1_HLs_Clim3['priority_substrate'] = "None"
for index, row in CLClim3NE1_HLs_Clim3.iterrows():
    if row['monomer_id'] in priority_substrates['id'].values:
        CLClim3NE1_HLs_Clim3.at[index, 'priority_substrate'] = priority_substrates[priority_substrates['id'] == row['monomer_id']]['protease_assignment'].values[0]
   
CLClim3NE1_HLs_Clim3

/var/folders/fg/mh5y1dgn2n5ft000p664m2z80000gq/T/ipykernel_52903/2850578870.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLClim3NE1_HLs_Clim3['priority_substrate'] = "None"


,monomer_id,degradation_rate(1/s)_y,half_life_(min)_y,degradation_rate_source_y,priority_substrate,common_name
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000023,501.707893,Gupta_et_al_MS_2024,None,plsC
1,1-PFK-MONOMER,0.000015,766.256894,Gupta_et_al_MS_2024,None,fruK
3,2-ISOPROPYLMALATESYN-MONOMER,0.000042,276.924332,Gupta_et_al_MS_2024,None,leuA
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000023,493.397050,Gupta_et_al_MS_2024,None,ubiE
5,2-OCTAPRENYLPHENOL-HYDROX-MONOMER,0.000015,796.038829,Gupta_et_al_MS_2024,None,ubiB
...,...,...,...,...,...,...
4288,YRBF-MONOMER,0.000014,823.833163,Gupta_et_al_MS_2024,None,mlaF
4290,YTFQ-MONOMER,0.000015,790.418789,Gupta_et_al_MS_2024,None,ytfQ
4291,YTFR-MONOMER,0.000018,654.149344,Gupta_et_al_MS_2024,None,ytfR
4293,ZNUA-MONOMER,0.000018,638.129701,Gupta_et_al_MS_2024,None,znuA


Add the common name to the data frame too:

In [30]:
common_names = "reconstruction/ecoli/scripts/protein_half_lives/Gupta_et_al_Clim_data/Clim_EcoCyc_monomer_ID_matches/41467_2024_49920_MOESM4_ESM_ST1_EcoCyc_monomer_ID_matches_11202024.tsv"
common_names = pd.read_csv(common_names, sep='\t', skiprows=1)

CLClim3NE1_HLs_Clim3['common_name'] = None
for index, row in CLClim3NE1_HLs_Clim3.iterrows():
    if row['monomer_id'] in common_names['Monomer ID'].values:
        CLClim3NE1_HLs_Clim3.at[index, 'common_name'] = common_names[common_names['Monomer ID'] == row['monomer_id']]['Common Name'].values[0]
   
CLClim3NE1_HLs_Clim3

/var/folders/fg/mh5y1dgn2n5ft000p664m2z80000gq/T/ipykernel_52903/566765461.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLClim3NE1_HLs_Clim3['common_name'] = None


,monomer_id,degradation_rate(1/s)_y,half_life_(min)_y,degradation_rate_source_y,priority_substrate,common_name
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000023,501.707893,Gupta_et_al_MS_2024,None,plsC
1,1-PFK-MONOMER,0.000015,766.256894,Gupta_et_al_MS_2024,None,fruK
3,2-ISOPROPYLMALATESYN-MONOMER,0.000042,276.924332,Gupta_et_al_MS_2024,None,leuA
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000023,493.397050,Gupta_et_al_MS_2024,None,ubiE
5,2-OCTAPRENYLPHENOL-HYDROX-MONOMER,0.000015,796.038829,Gupta_et_al_MS_2024,None,ubiB
...,...,...,...,...,...,...
4288,YRBF-MONOMER,0.000014,823.833163,Gupta_et_al_MS_2024,None,mlaF
4290,YTFQ-MONOMER,0.000015,790.418789,Gupta_et_al_MS_2024,None,ytfQ
4291,YTFR-MONOMER,0.000018,654.149344,Gupta_et_al_MS_2024,None,ytfR
4293,ZNUA-MONOMER,0.000018,638.129701,Gupta_et_al_MS_2024,None,znuA


Now, split up the substrates by protease assignment

In [35]:
# rename the columns in the CLClim3NE1_HLs_Clim3 data frame:
CLClim3NE1_HLs_Clim3 = CLClim3NE1_HLs_Clim3.rename(columns={"half_life_(min)_y": "half_life_(min)", "degradation_rate(1/s)_y": "degradation_rate(1/s)", "degradation_rate_source_y": "degradation_rate_source", "priority_substrate": "protease_assignment"})   

CLClim3NE1_HLs_Clim3_sorted = CLClim3NE1_HLs_Clim3.sort_values(by=["half_life_(min)"])
ordered_CLClimNE_Clim_10 = CLClim3NE1_HLs_Clim3_sorted.copy()

# figure out which proteins were assigned to lon in the model: 
lon_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'Lon only']
lon_proteins

# figure out which proteins were assigned to clp in the model:
clp_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'ClpP only']

hslv_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'HslV only']

# figure out which proteins are additive in the model: 
additive_proteins_LC = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'Additive: ClpP, Lon']

# figure out which proteins are additive in the model: 
additive_proteins_LCH = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'Additive: ClpP, Lon, HslV']

# figure out which proteins were not assigned to a protease in the model:
no_protease_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'None']
print()

# todo: note that these numbers will not add up to those in the graph below bc some proteins in the CLClim0NE model are not in the CLNE model 
#298 total 
print("Lon Proteins (" +str(len(lon_proteins))+"): ", list(lon_proteins['common_name'])) #6
print("ClpP Proteins (" +str(len(clp_proteins))+"): ", list(clp_proteins['common_name'])) #39
print("HslV Proteins (" +str(len(hslv_proteins))+"): ", list(hslv_proteins['common_name'])) #1
print("Additive (Lon + ClpP) Proteins (" +str(len(additive_proteins_LC))+"): ", list(additive_proteins_LC['common_name'])) #3
print("Additive (Lon + ClpP + HslV) Proteins (" +str(len(additive_proteins_LCH))+"): ", list(additive_proteins_LCH['common_name'])) #32

# figure out slices of the data:
ordered_CLClimNE_Clim_10_copy = ordered_CLClimNE_Clim_10.copy()
# make a copy of the ordered_df data:
CLClimNE_full_log_data_with_Lon_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_ClpP_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_HslV_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_Additive_LC_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_Additive_LCH_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_no_proteases = ordered_CLClimNE_Clim_10_copy.copy()

# remove the proteins that were not assigned to the protease in the model:
CLClimNE_full_log_data_with_Lon_proteases = CLClimNE_full_log_data_with_Lon_proteases[CLClimNE_full_log_data_with_Lon_proteases['protease_assignment'] == 'Lon only']
CLClimNE_full_log_data_with_ClpP_proteases = CLClimNE_full_log_data_with_ClpP_proteases[CLClimNE_full_log_data_with_ClpP_proteases['protease_assignment'] == 'ClpP only']
CLClimNE_full_log_data_with_HslV_proteases = CLClimNE_full_log_data_with_HslV_proteases[CLClimNE_full_log_data_with_HslV_proteases['protease_assignment'] == 'HslV only']
CLClimNE_full_log_data_with_Additive_LC_proteases = CLClimNE_full_log_data_with_Additive_LC_proteases[CLClimNE_full_log_data_with_Additive_LC_proteases['protease_assignment'] == 'Additive: ClpP, Lon']
CLClimNE_full_log_data_with_Additive_LCH_proteases = CLClimNE_full_log_data_with_Additive_LCH_proteases[CLClimNE_full_log_data_with_Additive_LCH_proteases['protease_assignment'] == 'Additive: ClpP, Lon, HslV']

priority_substrates_in_the_full_model = [CLClimNE_full_log_data_with_Lon_proteases, CLClimNE_full_log_data_with_ClpP_proteases, CLClimNE_full_log_data_with_HslV_proteases, CLClimNE_full_log_data_with_Additive_LC_proteases, CLClimNE_full_log_data_with_Additive_LCH_proteases]

# concatenate the dataframes:
priority_substrates_full_model_df = pd.concat(priority_substrates_in_the_full_model)
priority_substrates_full_model_df



Lon Proteins (6):  ['metR', 'nemA', 'ydcI', 'trxC', 'fhlA', 'rpoD']
ClpP Proteins (39):  ['phoH', 'rpoS', 'intA', 'clpA', 'patA', 'ribB', 'clpX', 'ftsZ', 'ydhQ', 'cysD', 'fadE', 'putA', 'nfo', 'dps', 'yfcZ', 'comR', 'exuR', 'glpD', 'dksA', 'lldD', 'obgE', 'parC', 'yheS', 'def', 'pcnB', 'yheO', 'rpoC', 'znuC', 'mukB', 'mdlB', 'uvrD', 'dnaB', 'mutS', 'rpsA', 'dnaK', 'sbcC', 'oxyR', 'rpoB', 'recA']
HslV Proteins (1):  ['uhpA']
Additive (Lon + ClpP) Proteins (3):  ['ibpA', 'grcA', 'dnaA']
Additive (Lon + ClpP + HslV) Proteins (32):  ['iscR', 'erpA', 'yibA', 'glaR', 'aroK', 'chaB', 'minE', 'yfhH', 'yaeP', 'proQ', 'ybaB', 'parE', 'yjgA', 'yiiQ', 'murQ', 'yihD', 'glnD', 'helD', 'mazF', 'ppiC', 'pspA', 'acpP', 'yggX', 'srmB', 'ycaR', 'rarA', 'ligA', 'rhlB', 'kbp', 'thiL', 'hprR', 'yiaU']


,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source,protease_assignment,common_name
3851,PD03938,0.000146,79.151002,Gupta_et_al_MS_2024,Lon only,metR
2334,G6890-MONOMER,0.000054,214.018094,Gupta_et_al_MS_2024,Lon only,nemA
2221,G6737-MONOMER,0.000040,288.738238,Gupta_et_al_MS_2024,Lon only,ydcI
3963,RED-THIOREDOXIN2-MONOMER,0.000031,367.223275,Gupta_et_al_MS_2024,Lon only,trxC
3842,PD02936,0.000025,468.602732,Gupta_et_al_MS_2024,Lon only,fhlA
...,...,...,...,...,...,...
633,EG10844-MONOMER,0.000018,624.526702,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",rhlB
2721,G7395-MONOMER,0.000018,635.846404,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",kbp
4058,THI-P-KIN-MONOMER,0.000018,656.519837,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",thiL
2467,G7057-MONOMER,0.000018,656.623355,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",hprR


Plot the half life values for proteins that were originally 10 hours in the CLNE1 model and are now assigned to a Clim3 half life in the CLClim3NE1 model

In [ ]:
# todo: figure out what is up with EG12298 because it used to have a HL of 7k mins or something!

In [40]:
# Clim3 HL plot!

fig = go.Figure()

# generate the figure in the background first:
fig = px.scatter(ordered_CLClimNE_Clim_10, x=ordered_CLClimNE_Clim_10["monomer_id"], y=ordered_CLClimNE_Clim_10["half_life_(min)"])
fig.update_traces(marker_size=.01, opacity=.01)    

# figure out slices of the data:
priority_protease_df = ordered_CLClimNE_Clim_10.copy()
# make a copy of the ordered_df data:
CLClimNE_Clim0_log_data_with_Lon_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_ClpP_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_HslV_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_Additive_LC_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_Additive_LCH_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_no_protease_proteases = priority_protease_df.copy()

# remove the proteins that were not assigned to the protease in the model:
CLClimNE_log_data_with_Lon_proteases = CLClimNE_Clim0_log_data_with_Lon_proteases[CLClimNE_Clim0_log_data_with_Lon_proteases['protease_assignment'] == 'Lon only']
CLClimNE_log_data_with_ClpP_proteases = CLClimNE_Clim0_log_data_with_ClpP_proteases[CLClimNE_Clim0_log_data_with_ClpP_proteases['protease_assignment'] == 'ClpP only']
CLClimNE_log_data_with_HslV_proteases = CLClimNE_Clim0_log_data_with_HslV_proteases[CLClimNE_Clim0_log_data_with_HslV_proteases['protease_assignment'] == 'HslV only']
CLClimNE_Clim0_log_data_with_Additive_LC_proteases = CLClimNE_Clim0_log_data_with_Additive_LC_proteases[CLClimNE_Clim0_log_data_with_Additive_LC_proteases['protease_assignment'] == 'Additive: ClpP, Lon']
CLClimNE_Clim0_log_data_with_Additive_LCH_proteases = CLClimNE_Clim0_log_data_with_Additive_LCH_proteases[CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['protease_assignment'] == 'Additive: ClpP, Lon, HslV']
CLClimNE_log_data_with_no_protease_proteases = CLClimNE_Clim0_log_data_with_no_protease_proteases[CLClimNE_Clim0_log_data_with_no_protease_proteases['protease_assignment'] == 'None']

# overlay the individual rates
fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_no_protease_proteases['monomer_id']), y=(CLClimNE_log_data_with_no_protease_proteases["half_life_(min)"]), mode='markers', name=("No assignment"+ ' '+ (str(np.shape(CLClimNE_log_data_with_no_protease_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_no_protease_proteases['common_name'], marker_size=2, marker=dict(color="lightseagreen", size=3, opacity=.2), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_Clim0_log_data_with_Additive_LC_proteases['monomer_id']), y=(CLClimNE_Clim0_log_data_with_Additive_LC_proteases["half_life_(min)"]), mode='markers', name=("Additive: ClpP, Lon "+ (str(np.shape(CLClimNE_Clim0_log_data_with_Additive_LC_proteases['monomer_id'])[0]))), hovertext=CLClimNE_Clim0_log_data_with_Additive_LC_proteases['common_name'], marker_size=4, marker=dict(color='mediumturquoise', size=3, opacity=1), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['monomer_id']), y=(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases["half_life_(min)"]), mode='markers', name=("Additive: ClpP, Lon, HslV "+ (str(np.shape(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['monomer_id'])[0]))), hovertext=CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['common_name'], marker_size=4, marker=dict(color='orange', size=3, opacity=.6), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_ClpP_proteases['monomer_id']), y=(CLClimNE_log_data_with_ClpP_proteases["half_life_(min)"]), mode='markers', name=("ClpP  "+ (str(np.shape(CLClimNE_log_data_with_ClpP_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_ClpP_proteases['common_name'], marker_size=3, marker=dict(color='deeppink', size=3, opacity=.4), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_Lon_proteases['monomer_id']), y=(CLClimNE_log_data_with_Lon_proteases["half_life_(min)"]), mode='markers', name=("Lon  "+ (str(np.shape(CLClimNE_log_data_with_Lon_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_Lon_proteases['common_name'], marker_size=5, marker=dict(color="yellowgreen", size=3, opacity=.8), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_HslV_proteases['monomer_id']), y=(CLClimNE_log_data_with_HslV_proteases["half_life_(min)"]), mode='markers', name=("HslV  "+ (str(np.shape(CLClimNE_log_data_with_HslV_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_HslV_proteases['common_name'], marker_size=5, marker=dict(color="brown", size=3, opacity=.6), yaxis="y1"))
            
            
# Plot Specs
fig.update_layout(title=dict(text=f'The C-limited (Clim) half life values for the New WCM (CLClim3NE1) proteins assigned to Clim3 <br> half-lives that were assinged to a half-life of 10 hours in the original model (CLNE1) ', font=dict(size=15)), xaxis_title='Protein ID', yaxis_title='Clim Half Life (min)')
fig.update_xaxes(visible=False)
fig.update_layout(autosize=False, width=900, height=600, showlegend=True)

# wont open in pycharm, so save as a html:
out_pth = "models/ecoli/analysis/local_notebooks/protease_substrate_relationships/CLClim3NE1_Clim3_HLs_that_were_600_in_CLNE1.html" 
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web browser 
